In [1]:
import numpy as np
import healpy as hp
import pickle
from matplotlib.pyplot import *
# cmblensplus
import analysis as ana
import basic
import curvedsky
# local
import prjlib

In [3]:
h = 6.626e-34
c = 3e8
kB = 1.38e-23

def Int_Planck(nu,T): 
    '''
    Black body I(nu)
    nu [GHz]
    '''
    return 2*h*(nu*1e9)**3/c**2 * (np.exp(h*nu*1e9/kB/T)-1.)**(-1)

def Int_dust(nu,Td=19.6,beta=1.53): 
    '''
    Dust I(nu) as a modified Black body
    Td, beta: values from arXiv 1801.04945
    nu [GHz]
    '''
    return (nu*1e9)**beta*Int_Planck(nu,Td)

#def RJfunc(nu,T):
#    return 2*nu**2/c**2 * kB*T
#
#def Int_Planck_deriv(nu,T):
#    return 2*h*nu**3/c**2 * (np.exp(h*nu/kB/T)-1.)**(-2)*(np.exp(h*nu/kB/T)*h*nu/kB/T**2)

In [4]:
R = Int_dust(150)/Int_dust(353)
C = 283./372. # MJy/sr <-> K_CMB conversion factor (avg)
#Tcmb = 2.726
#V0 = Int_Planck_deriv(150*1e9,Tcmb)#*Tcmb/RJfunc(150*1e9,Tcmb)
#V1 = Int_Planck_deriv(353*1e9,Tcmb)#*Tcmb/RJfunc(353*1e9,Tcmb)
print('total scaliing',R*C,R,C)

total scaliing 0.0487746935452379 0.06411373144462366 0.760752688172043


In [ ]:
ps, fs = prjlib.filename_init(doreal='False',dodust='False')
pd, fd = prjlib.filename_init(doreal='False',dodust='True')

In [ ]:
Tgal = hp.fitsfunc.read_map(fs.dust,field=0)*R*C/2.726
Qgal = hp.fitsfunc.read_map(fs.dust,field=1)*R*C/2.726
Ugal = hp.fitsfunc.read_map(fs.dust,field=2)*R*C/2.726
M = hp.fitsfunc.read_map(fs.amask)
rM = hp.fitsfunc.read_map(fs.rmask)
W2 = np.average(rM**2*M**2)
w2 = np.average(M**2)
print(W2,w2)

In [ ]:
T = ana.change_coord(Tgal,coord=['G','C'])
Q = ana.change_coord(Qgal,coord=['G','C'])
U = ana.change_coord(Ugal,coord=['G','C'])

In [ ]:
v = 1e-5
hp.mollview(Q,min=0,max=v)
hp.mollview(U,min=0,max=v)

In [ ]:
l, EEs, BBs = np.loadtxt(fs.scl,usecols=(0,2,3),unpack=True)
icls = basic.aps.read_cambcls(fs.lcl,2,3000,4,bb=True)/(2.72e6)**2
Tl = np.zeros(3001)
Tl[20:] = np.sqrt((EEs[20:]-BBs[20:])/icls[1,20:])
xlim(2,2048)
plot(l[20:],Tl[20:])

In [ ]:
Tlm = curvedsky.utils.hp_map2alm(2048,ps.lmax,ps.lmax,T*M)
alm = curvedsky.utils.hp_map2alm_spin(2048,ps.lmax,ps.lmax,2,Q*M,U*M)
dElm = alm[0,:,:]*Tl[:,None]
dBlm = alm[1,:,:]*Tl[:,None]
sElm = pickle.load(open(fs.alm['E'][0],"rb"))
sBlm = pickle.load(open(fs.alm['B'][0],"rb"))
tElm = sElm + dElm
tBlm = sBlm + dBlm

In [ ]:
L = np.linspace(0,ps.lmax,ps.lmax+1)
#TT = curvedsky.utils.alm2cl(p.lmax,Tlm)#/w2
EE = curvedsky.utils.alm2cl(ps.lmax,dElm)/w2
BB = curvedsky.utils.alm2cl(ps.lmax,dBlm)/w2
ee = curvedsky.utils.alm2cl(ps.lmax,alm[0,:,:])/w2
bb = curvedsky.utils.alm2cl(ps.lmax,alm[1,:,:])/w2
#EB = curvedsky.utils.alm2cl(ps.lmax,dElm,dBlm)/w2
sEE = curvedsky.utils.alm2cl(ps.lmax,sElm)/w2
sBB = curvedsky.utils.alm2cl(ps.lmax,sBlm)/w2
tEB = curvedsky.utils.alm2cl(ps.lmax,tElm,tBlm)/w2
l, mTT, mEE, mBB = (np.loadtxt(fs.scl,usecols=(0,1,2,3))).T
s = 2.72e6**2*L*(L+1)/2/np.pi
xlim(100,1000)
grid(True)
axvline(200,ls='--',color='k')
ylim(1e-2,1e2)
#loglog(L,TT*s)
loglog(L,EE*s)
loglog(L,BB*s)
loglog(L,ee*s)
loglog(L,bb*s)
loglog(L,sEE*s,label='s')
loglog(L,sBB*s,label='s')
legend(loc=0)

In [ ]:
pickle.dump((tElm),open(fd.alm['E'][0],"wb"),protocol=pickle.HIGHEST_PROTOCOL)
pickle.dump((tBlm),open(fd.alm['B'][0],"wb"),protocol=pickle.HIGHEST_PROTOCOL)